# Correlation statistics for regional variables

This notebook calculates slope, p value, standard deviation of the slope, and correlation coefficients for the linear and exponential fit

Loading libraires:

In [5]:
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy import stats
import os

Define functions:

In [6]:
def read_txt(fname, ncolumns, dtype):
    fdata   = open(fname)
    flines  = np.array(fdata.readlines())
    data_in = np.empty([len(flines)])
    if dtype == 'string':
        data_in = np.empty([len(flines)], dtype=object)
    for i in np.arange(len(flines)):
        data_in[i]=flines[i].split()[ncolumns-1]      
    return(data_in)
    fdata.close()
    
# Define functions
def read_data(fname, ncolumns):
    if ncolumns > 1:
        fdata   = open(fname)
        flines  = np.array(fdata.readlines())
        data_in = np.empty([len(flines), ncolumns])
        for i in np.arange(len(flines)):
            data_in[i,:]=flines[i].split()
    else:
        with open(fname) as flist:
            data_in = np.array(flist.readlines())        
    return(data_in)
    fdata.close()

def read_nc(fname, varname, ifmerged):
    if ifmerged:
        f  = nc.MFDataset(fname)
    else:
        f  = nc.Dataset(fname)
    var    = f.variables[varname]
    lon    = f.variables['lon']
    lat    = f.variables['lat']
    time   = f.variables['time']
    dates  = nc.num2date(time[:], time.units, time.calendar)
    return var, lon, lat, time, dates

def read_mask(fname, varname):
    f = nc.Dataset(fname)
    var  = f.variables[varname][:]
    return var

Define enviromental variables and paths:

In [19]:
resolution   = 'low_res'
seasons      = ['ANNUAL', 'DJF', 'MAM', 'JJA', 'SON'] 
mask_file    = 'regions.nc' # 'biomes_mask.nc' 'regions.nc'
member_names = f'models_{resolution}.txt'
varname      = 'stas' # tas tos
ssps         = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
nssps        = len(ssps)
ref_ntimes   = 51
ystart_to_plot  = 2010
ystart = int(ystart_to_plot - 1950)

root = '..'
path2data = f'{root}/data/data_txt/'  
path2info  = f'{root}/data/data_info/'

Define mask variable for corresponding masks:

In [20]:
if mask_file == 'biomes_mask.nc':
    maskvar = 'sftlf'
    mask_values  = [0,1,2,3,4,5,6,7,8,9,10]         
    dirname = 'biomes_regs'
    
elif mask_file == 'regions.nc':
    maskvar = 'regionID'
    mask_values  = [20,47,48,49,50,51,52,53,54,55,56,57,58]      
    dirname = 'ipcc_regs'
    
nregions = len(mask_values)

Read list of the models to be processed:

In [21]:
member_list = f'{path2info}/{member_names}'
members  = read_txt(member_list, 1, 'string')
nmembers = len(members)

Read list the names of the regions of to be processed:

In [22]:
regions_list = f'{path2info}/{dirname}.txt'
reg_names  = read_txt(regions_list, 2, 'string')
nregions = len(reg_names)

Main code:

In [ ]:
# Loop over seasons
for season in seasons:
        print(f'Working on the season: {season}')

        Beta   = np.empty((nregions, nssps),dtype=float)
        R2_lin = np.empty((nregions, nssps),dtype=float)
        R2_exp = np.empty((nregions, nssps),dtype=float)
        p_value = np.empty((nregions, nssps),dtype=float)
        std_value = np.empty((nregions, nssps),dtype=float)

        for s in np.arange(nssps):
            ssp = ssps[s]
            print("        Working on the ssp: " + ssp)          

            # Read reference data
            ref_fname = f'{path2data}/{resolution}/global/tas/ANNUAL/tas_{ssp}_{season}.csv'
            ref_data = read_data(ref_fname, nmembers)

            # Define size of the panel plot
            for r in np.arange(nregions):
                region=mask_values[r]
                reg_name=reg_names[r]  

                dirpath  = f'{resolution}/regional/{dirname}/{varname}/{season}' 
                filename = f'{varname}_{ssp}_reg{region}_{season}'
                infile   = f'{path2data}/{dirpath}/{filename}.csv'
                data     = read_data(infile, nmembers)

                # Calculate and plot linreadear fit                   
                x = ref_data[ystart::,:].reshape(-1, 1)
                y = data[ystart::,:].reshape(-1, 1)
                Beta[r,s], intercept, rvalue, p_value[r,s], std_value[r,s] = stats.linregress(x[:,0], y[:,0])
                R2_lin[r,s] = rvalue*rvalue  

                # Calculate and plot exponential fit
                def func(x, a, b): return a * np.exp(b * x)
                popt, pcov = curve_fit(func, x[:,0], y[:,0])     # a = popt[0], b = popt[1]
                x_fitted = np.linspace(np.min(x[:]), np.max(x[:]), 100)
                y_fitted = func(x_fitted, *popt)                 #a * np.exp(b * x_fitted)

                # Calculate R2 for the exponential fit
                residuals = y[:,0] - func(x[:,0], *popt)
                res = np.sum(residuals**2)
                tot = np.sum((y[:,0]-np.mean(y[:,0]))**2)
                R2_exp[r,s] = 1 - (res / tot)

        # Save statistical info in a txt file
        print("    Saving the data in the output file.")
        path2out  = f'{path2data}/{resolution}/regional/{dirname}/{varname}/{season}'
        if not os.path.exists(path2out):
            os.makedirs(path2out)            

        np.savetxt(f'{path2out}/Beta_{varname}_{season}.csv',      Beta,      delimiter =" ",fmt='%6.2f')
        np.savetxt(f'{path2out}/R2lin_{varname}_{season}.csv',     R2_lin,    delimiter =" ",fmt='%6.2f')
        np.savetxt(f'{path2out}/R2exp_{varname}_{season}.csv',     R2_exp,    delimiter =" ",fmt='%6.2f')
        np.savetxt(f'{path2out}/p_value_{varname}_{season}.csv',   p_value,   delimiter =" ",fmt='%6.5f')
        np.savetxt(f'{path2out}/std_value_{varname}_{season}.csv', std_value, delimiter =" ",fmt='%6.5f')